---

## Audio File Processing to Remove Silence and Segment into Clips

*Author: Rohit*

### Overview

The code provides a utility for processing audio files in a directory by removing silent portions and subsequently segmenting the non-silent parts into 2-second clips. It uses the `pydub` library to achieve this, which offers simple and intuitive audio manipulation capabilities.

### Components

#### Dependencies
- `AudioSegment`: A class from `pydub` that enables easy audio file manipulation.
- `split_on_silence`: A function from `pydub` that segments audio based on silent durations.
- `os`: Python's built-in module for interacting with the operating system, useful for directory and file operations.

#### Function: `process_audio_files`

This function serves as the core of the utility, performing the desired audio processing tasks. It accepts four parameters:

- `input_dir`: Directory containing the raw .wav audio files.
- `output_dir`: Directory where the processed audio files will be saved.
- `silence_len`: The minimum duration (in milliseconds) of silence to consider for splitting the audio.
- `silence_thresh`: The threshold (in dB) below which audio is considered silent.

##### Workflow:

1. **Setup**: Ensure the output directory exists, or create it.
2. **Iterate through audio files**: It goes through each .wav file in the input directory and its subdirectories.
3. **Silence-based segmentation**: The audio file is loaded, and silent portions are identified. Using `split_on_silence`, the file is split at these silent portions.
4. **Segment saving**: The non-silent segments (chunks) are then further divided into 2-second audio files and saved with appropriate naming conventions. If there's a remaining segment shorter than 2 seconds but longer than 1 second, it's also saved.

#### Main Execution

The script, when run as the main module, specifies the input and output directories and invokes the `process_audio_files` function to start the processing.

### Sample Usage:

```python
# Specify input and output directories
input_directory = '/path/to/raw_audio_files'
output_directory = '/path/to/processed_audio_files'

# Call the function
process_audio_files(input_directory, output_directory)
```

### Conclusion

This script simplifies the task of segmenting large audio files into smaller clips while discarding silent portions. It's particularly useful in data preparation stages of audio analysis projects where silence removal can lead to more focused and efficient analyses.

---

You can adapt this documentation according to your specific requirements or context.

In [14]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os

def process_audio_files(input_dir, output_dir, silence_len=500, silence_thresh=-40):
    """
    Process audio files to remove silence and segment them into 2-second clips.
    
    Args:
    input_dir (str): The directory containing the audio files.
    output_dir (str): The directory to save the processed audio files.
    silence_len (int): The minimum length of silence to be used as a split point (in milliseconds).
    silence_thresh (int): The silence threshold (in dB) to be used as a split point.
    """

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Iterate over all subdirectories in the input directory
    for subdir, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.wav'):
                # Create a corresponding subdirectory in the output directory
                output_subdir = os.path.join(output_dir, os.path.relpath(subdir, input_dir))
                if not os.path.exists(output_subdir):
                    os.makedirs(output_subdir)
                
                # Load the audio file
                filepath = os.path.join(subdir, file)
                audio = AudioSegment.from_wav(filepath)
                
                # Split the audio file on silence
                chunks = split_on_silence(audio, min_silence_len=silence_len, silence_thresh=silence_thresh)
                
                # Save each non-silent chunk as a separate 2-second audio file
                segment_duration = 2000  # Define the segment duration as 2000 ms (2 seconds)
                for i, chunk in enumerate(chunks):
                    j = 0  # Initialize the segment start index
                    while j + segment_duration < len(chunk):
                        segment = chunk[j:j+segment_duration]
                        output_filepath = os.path.join(output_subdir, f"{os.path.splitext(file)[0]}_chunk-{i}_segment-{j//segment_duration}.wav")
                        segment.export(output_filepath, format="wav")
                        j += segment_duration  # Move to the next segment
                    
                    # Handle the remaining short segment
                    remaining_segment = chunk[j:]
                    if len(remaining_segment) >= segment_duration / 2:  # Save if remaining segment is longer than 1 second
                        output_filepath = os.path.join(output_subdir, f"{os.path.splitext(file)[0]}_chunk-{i}_segment-remaining.wav")
                        remaining_segment.export(output_filepath, format="wav")


if __name__ == "__main__":
    # Specify the input and output directories
    input_dir = '/Users/ankush/Downloads/deakin-units/data/b3'
    output_dir = '/Users/ankush/Downloads/deakin-units/data/cleaned3'

    # Process the audio files
    process_audio_files(input_dir, output_dir)
